In [1]:
#!/usr/bin/env python3
import scipy
import scipy.stats
import numpy as np
import nilearn.plotting
import nilearn
import nibabel
import sys
import os
import glob
import argparse
import toleranceinterval as ti
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)

NaN = float('nan')
preproc_extension = '_desc-preproc_T1w.nii.gz'
brain_mask_extension = '_desc-brain_mask.nii.gz'

In [2]:
def error(msg):
    print(msg, file=sys.stderr)
    sys.exit(1)
    
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def print_info(args, pass_test, pass_test_msg, ratio):
    if args.template == 'MNI152NLin6Asym':
        template = f'{bcolors.OKBLUE}{args.template:24}{bcolors.ENDC}'
    else:
        template = f'{bcolors.OKCYAN}{args.template:24}{bcolors.ENDC}'

    if pass_test:
        header = f'{bcolors.OKGREEN}{pass_test_msg:^7}{bcolors.ENDC}'
        succ = bcolors.OKGREEN
    else:
        header = f'{bcolors.FAIL}{pass_test_msg:^7}{bcolors.ENDC}'
        succ = bcolors.FAIL

    dataset = f'{bcolors.WARNING}{args.dataset:10}{bcolors.ENDC}'
    subject = f'{bcolors.WARNING}{args.subject:16}{bcolors.ENDC}'
    datatype = f'{bcolors.HEADER}{args.data_type}{bcolors.ENDC}'

    ratio = f'{bcolors.BOLD}{succ}{100*ratio:.2f}%{bcolors.ENDC}'

    info = f'dataset:{dataset} subject:{subject} datatype:{datatype} template:{template} target:{args.target} '
    print(f"[{header}] {ratio} {info}")
    

In [3]:
def load_image(path):
    return nibabel.load(path)

In [4]:
def get_reference(args):
    '''
    Gather images used as reference
    '''
    data = []
    preproc_re = f'{args.subject}*{args.template}{preproc_extension}'
    mask_re = f'{args.subject}*{args.template}{brain_mask_extension}'
    regexp = os.path.join(
        args.reference, f'*{args.dataset}*', 'fmriprep', args.subject, args.data_type)
    paths = glob.glob(regexp)
    for path in paths:
        image = load_image(glob.glob(os.path.join(path, preproc_re))[0])
        mask = load_image(glob.glob(os.path.join(path, mask_re))[0])
        image_masked = np.ma.where(
            mask.get_fdata().astype('bool'), image.get_fdata(), False)
        data.append(image_masked)
    return np.array(data)


def get_target(args):
    '''
    Gather images used as target
    '''
    data = []
    preproc_re = f'{args.subject}*{args.template}{preproc_extension}'
    mask_re = f'{args.subject}*{args.template}{brain_mask_extension}'
    regexp = os.path.join(
        args.target, f'*{args.dataset}*', 'fmriprep', args.subject, args.data_type)
    paths = glob.glob(regexp)
    for path in paths:
        image = load_image(glob.glob(os.path.join(path, preproc_re))[0])
        mask = load_image(glob.glob(os.path.join(path, mask_re))[0])
        image_masked = np.ma.where(
            mask.get_fdata().astype('bool'), image.get_fdata(), False)
        data.append(image_masked)
    return np.array(data)


def get_mean_reference(reference):
    return np.ma.mean(reference, axis=0, dtype=np.float64)

def c4(n):
    '''    
    c4(n) = sqrt(2/n-1) (gamma(n/2)/gamma(n-1/2))
    '''
    gamma = scipy.special.gamma
    return np.sqrt(2/(n-1)) * (gamma(n/2)/gamma((n-1)/2))

def get_std_reference(reference):
    '''
    Unbiased estimator for standard deviation with small sample size.
    '''
    return np.ma.std(reference, axis=0, ddof=1, dtype=np.float64) / c4(reference.shape[0])

In [5]:
def compute_prediction_interval(mean, std, value, n, confidence):
    '''
    Compute prediction interval
    '''
    alpha = 1 - confidence
    coef = scipy.stats.t.ppf(1-alpha/2, df=n-1)
    bounds_inf = mean - coef * np.sqrt((1+1/n)) * std
    bounds_sup = mean + coef * np.sqrt((1+1/n)) * std
    success = np.ma.logical_and(bounds_inf <= value, value <= bounds_sup)
    return success

def compute_k2(n, confidence, population):
    '''
    Factor for tolerance interval under normal hypothesis
    '''
    return ti.twoside.normal_factor(n,  population, confidence)


def compute_tolerance_interval(mean, std, value, n, confidence, population):
    '''
    Compute tolerance interval
    '''
    coef = compute_k2(n, confidence, population)
    bounds_inf = mean - abs(coef) * std
    bounds_sup = mean + coef * std
    success = np.ma.logical_and(bounds_inf <= value, value <= bounds_sup)
    return success


def compute_test(mean, std, target, n, confidence, population):
    '''
    Compute the test. If only confidence is given, computes the prediction interval.
    If confidence and population are given, computes the tolerance interval.
    '''
    if population:
        success = compute_tolerance_interval(
            mean, std, target, n, confidence, population)
    else:
        success = compute_prediction_interval(
            mean, std, target, n, confidence)

    nb_success = np.ma.sum(success)
    nb_voxels = np.ma.count(success)
    return success, nb_success, nb_voxels

In [20]:
def test(reference, target, confidence, population):
    '''
    Compute the test. if target or reference is missing, returns NaN.
    '''
    if target.size == 0 or reference.size == 0:
        return None,NaN,NaN
    
    n = reference.shape[0]
    mean = get_mean_reference(reference)
    std = get_std_reference(reference)
    return compute_test(target=target,
                        mean=mean,
                        std=std,
                        confidence=confidence,
                        n=n,
                        population=population)

def test_against_sample(args):
    '''
    Test the sample with itself.
    Let X the sample made of N observation X_k, 0<k<N.
    This function uses one observation X_i as target 
    and computes the SI with the last N-1 X_j observations, 0<j<N, i!=j.
    Do this for each i < N.    
    '''
    confidence = args.confidence
    population = args.population
    sample = get_reference(args)
    sample_size = sample.shape[0]

    pass_test = True

    for i, observation in enumerate(sample):
        index = list(range(sample_size))
        index.pop(i)
        reference = sample[index]
        target = observation
        success, nb_success, nb_voxels = test(reference, target, confidence, population)
        ratio = nb_success/nb_voxels
        _pass_test = ratio >= args.confidence
        pass_test_msg = 'Success' if _pass_test else 'Fail'
        print_info(args, _pass_test, pass_test_msg, ratio)
        pass_test = pass_test and _pass_test

    return pass_test


def test_against_reference(args):
    '''
    Test that the target is in the SI computed with the reference.
    '''    
    confidence = args.confidence
    population = args.population
    reference = get_reference(args)
    targets = get_target(args)
    pass_test = True

    for target in targets:    
        success, nb_success, nb_voxels = test(
            reference, target, confidence, population)
        ratio = nb_success/nb_voxels
        _pass_test = ratio >= args.confidence
        pass_test_msg = 'Success' if _pass_test else 'Fail'
        print_info(args, _pass_test, pass_test_msg, ratio)
        pass_test = pass_test and _pass_test

    return pass_test

In [7]:
def parse_args(sargs):
    parser = argparse.ArgumentParser(
        description="Test target image is in a confidence interval"
    )
    parser.add_argument(
        "--confidence", action="store", default=0.95, type=float, help="Confidence"
    )
    parser.add_argument(
        "--population", action="store", type=float, help="Population"
    )
    parser.add_argument('--reference', action='store',
                        required=True, help="Reference directory")
    parser.add_argument("--target", action="store",
                        help="Target directory (if empty, target is reference)")
    parser.add_argument("--dataset", action="store",
                        required=True, help="Dataset")
    parser.add_argument("--subject", action="store",
                        required=True, help="Subject")
    parser.add_argument("--data-type", action="store",
                        required=True, help="Data type")
    parser.add_argument("--template", action="store",
                        required=True, help="Template")

    args = parser.parse_args(sargs)

    return args

In [8]:
def main(sargs):
    args = parse_args(sargs)
    if args.target is None:
        pass_test = test_against_sample(args)
    else:
        pass_test = test_against_reference(args)

In [9]:
def run(confidence, population, reference, target, dataset, subject, data_type, template):
    sargs = [f'--confidence={confidence}',
             f'--reference={reference}',
             f'--dataset={dataset}',
             f'--subject={subject}',
             f'--data-type={data_type}',
             f'--template={template}']
    sargs += [f'--population={population}'] if population is not None else []
    sargs += [f'--target={target}'] if target is not None else []
    
    main(sargs)

In [10]:
def run_all(confidence, population, reference, target, datasets, data_type, templates):
    for template in templates:
        for dataset, subjects in datasets.items():
            for subject in subjects.keys():
                run(confidence, population, reference, target, dataset, subject, data_type, template)

In [11]:
fi = open('fmriprep-reproducibility/fmriprep-cmd.json')
datasets = json.load(fi)
data_type = 'anat'
templates = ['MNI152NLin2009cAsym','MNI152NLin6Asym']
confidence = 0.95
population = 0.95

In [12]:
reference='outputs/fuzzy'
target='outputs/ieee'
run_all(confidence, population, reference, target, datasets, data_type, templates)        

[Success] 99.88% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.73% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.97% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.71% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.48% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.00% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.94% dataset:ds001600   subject:sub-1            datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.93% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152

In [25]:
reference='outputs/fuzzy'
target='outputs/ieee_update'
run_all(confidence, population, reference, target, datasets, data_type, templates)        

[ Fail  ] 92.46% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 99.61% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 97.20% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[ Fail  ] 94.07% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 96.94% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 95.25% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 97.45% dataset:ds001600   subject:sub-1            datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 96.39% dataset:ds001771   subje

In [26]:
reference='outputs/fuzzy'
target='outputs/ieee_seed'
run_all(confidence, population, reference, target, datasets, data_type, templates)        

[Success] 99.83% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.59% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.64% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.25% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.62% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 97.94% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.93% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_seed 
[Success] 99.23% dataset:ds001748   subject:sub-adult15

[Success] 99.71% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 99.93% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 99.33% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 99.66% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 98.92% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 98.36% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 99.30% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:outputs/ieee_seed 
[Success] 99.77% dataset:ds002338   subject:sub-xp201  

In [27]:
reference='outputs/ieee_seed'
target='outputs/ieee'
run_all(confidence, population, reference, target, datasets, data_type, templates)

[Success] 99.59% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.85% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.86% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.73% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.86% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[Success] 99.96% dataset:ds001600   subject:sub-1            datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee 
[ Fail  ] nan% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin

In [28]:
reference='outputs/ieee_seed'
target='outputs/ieee_update'
run_all(confidence, population, reference, target, datasets, data_type, templates)

[ Fail  ] 93.13% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 96.24% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[ Fail  ] 94.38% dataset:ds001748   subject:sub-adult16      datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 97.38% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 96.89% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[Success] 99.18% dataset:ds001600   subject:sub-1            datatype:anat template:MNI152NLin2009cAsym      target:outputs/ieee_update 
[ Fail  ] nan% dataset:ds001771   subject:s

In [29]:
reference='outputs/ieee_seed'
target='outputs/fuzzy'
run_all(confidence, population, reference, target, datasets, data_type, templates)

[Success] 99.76% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[Success] 99.47% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[Success] 99.53% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[Success] 99.62% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[Success] 99.59% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[Success] 99.64% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:outputs/fuzzy 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI

[Success] 99.90% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[Success] 99.85% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[Success] 99.81% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[Success] 99.83% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[Success] 99.81% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[Success] 99.70% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin6Asym          target:outputs/fuzzy 
[ Fail  ] nan% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI

In [30]:
reference='outputs/fuzzy'
target=None
run_all(confidence, population, reference, target, datasets, data_type, templates)        

[Success] 99.82% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.80% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.86% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.89% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.71% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.87% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.30% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.74% dataset:ds000256   subject:sub-CTS210       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.76% dataset

[Success] 98.15% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.91% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 98.01% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.74% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.90% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.87% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.83% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.80% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.72% dataset

[Success] 99.95% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.85% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.64% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.76% dataset:ds001771   subject:sub-36           datatype:anat template:MNI152NLin6Asym          target:None 


In [31]:
reference='outputs/ieee_seed'
target=None
run_all(confidence, population, reference, target, datasets, data_type, templates)        

[Success] 99.87% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.72% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.78% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.78% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.81% dataset:ds000256   subject:sub-CTS201       datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.77% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 98.83% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.90% dataset:ds001748   subject:sub-adult15      datatype:anat template:MNI152NLin2009cAsym      target:None 
[Success] 99.89% dataset

[Success] 99.83% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.71% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.76% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.55% dataset:ds002338   subject:sub-xp207        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.84% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.91% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.82% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 97.40% dataset:ds002338   subject:sub-xp201        datatype:anat template:MNI152NLin6Asym          target:None 
[Success] 99.83% dataset